In [1]:
# imports
import requests
from IPython.display import JSON
import pandas as pd
import os

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [2]:
#ll contains the latitude and longitude for all the bikestations in Vancouver.
df = pd.read_csv("bike_info.csv")
rows = df.shape[0]
ll = []
for i in range(rows):
    ll.append((f"{str(df['latitude'][i])},{str(df['longitude'][i])}"))

In [3]:
def station(ll_value):
    '''
    ll_value: Takes in a string with "latitude,longitude"
    return: HTTP Code (200 for success)
    '''
    headers = {"Accept": "application/json"}
    headers["Authorization"] = os.environ.get('FOURSQUARE_API_KEY')
    fields = 'name,rating,categories,price,popularity,geocodes,distance,features'
    url = (f'https://api.foursquare.com/v3/places/search')
    parameters = {'ll' : ll_value,
                  'fields' : fields,
                  'radius' : 1000,
                  'categories' : '13065,13003',
                  'limit' : '50'
                 }
    four_square_networks = requests.get(url, headers=headers, params=parameters)
    return four_square_networks

In [4]:
JSON(station("49.262487,-123.114397").json())

<IPython.core.display.JSON object>

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [5]:
def POI_fs(request):
    '''
    request: HTTP Code <200>
    Take in a HTTP code and convert it to JSON.
    Return: List
    '''
    request_in_json = request.json()
    results = request_in_json['results']
    list_of_results = []
    for result in results:
        new_dict = {}
        new_dict['name'] = result.get('name', None)
        new_dict['popularity'] = result.get('popularity', None)
        new_dict['price'] = result.get('price', None)
        new_dict['rating'] = result.get('rating', None)
        new_dict['categories_name'] = result.get('categories', {})[0].get('name', None)
        new_dict['latitude'] = result.get('geocodes', {}).get('main', {}).get('latitude', None)
        new_dict['longitude'] = result.get('geocodes', {}).get('main', {}).get('longitude', None)
        new_dict['distance'] = result.get('distance', None)
        new_dict['credit_card'] = result.get('features', {}).get('payment', {}).get('credit_cards', {}).get('accepts_credit_cards', None)
        new_dict['beer'] = result.get('features', {}).get('food_and_drink', {}).get('alcohol', {}).get('beer', None)
        new_dict['outdoor_seating'] = result.get('features', {}).get('amenities', {}).get('outdoor_seating', None)
        
        list_of_results.append(new_dict)
    return list_of_results

Put your parsed results into a DataFrame

In [6]:
def data_frame(listOfResults):
    '''
    listOfResults: list -> DataFrame
    '''
    return pd.DataFrame(listOfResults)

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [7]:
def yelp_station(ll_value):
    '''
    ll_value: str
    return: HTTPCode
    Take a string that looks like "latitude,longitude"
    '''
    headers_yelp = {}
    headers_yelp["Authorization"] = "Bearer " + os.environ.get('YELP_API_KEY')
    url = "https://api.yelp.com/v3/businesses/search"
    parameters = {'latitude' : ll_value.split(",")[0],
                  'longitude' : ll_value.split(",")[1],
                  'radius': 1000,
                  'categories' : "bars,restaurants",
                  'limit': 50,
                 }
    yelp_networks = requests.get(url, headers=headers_yelp, params=parameters)
    return yelp_networks

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [8]:
def POI_yelp(request):
    '''
    request: HTTP Code <200>
    Take in a HTTP code and convert it to JSON.
    Return: List
    '''
    request_in_json = request.json()
    businesses = request_in_json['businesses']
    list_of_businesses = []
    for business in businesses:
        new_dict = {}
        new_dict['name'] = business.get('name', None)
        new_dict['price'] = business.get('price', None)
        new_dict['rating'] = business.get('rating', None)
        new_dict['latitude'] = business.get('coordinates', None).get('latitude', None)
        new_dict['longtitude'] = business.get('coordinates', None).get('longitude', None)
        new_dict['review_count'] = business.get('review_count', None)
        new_dict['distance'] = business.get('distance', None)
        list_of_businesses.append(new_dict)
    return list_of_businesses

Put your parsed results into a DataFrame

In [9]:
def df_yelp(poi_yelp):
    return pd.DataFrame(poi_yelp)

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating

In [10]:
def mainfunction_foursq(llvalue):
    request_code = station(llvalue)
    point = POI_fs(request_code)
    return data_frame(point)

print(mainfunction_foursq("49.262487,-123.114397").sort_values(by='rating', ascending=False).head(10))

                       name  popularity  price  rating    categories_name  \
7       33 Acres Brewing Co    0.957773    2.0     9.2                Bar   
9            Elysian Coffee    0.951765    1.0     8.9               Café   
4             Milano Coffee    0.954459    2.0     8.8        Coffee Shop   
1    Rogue Kitchen & Wetbar    0.980815    3.0     8.8                Bar   
6                     Vij's    0.958393    2.0     8.8  Indian Restaurant   
0   Hokkaido Ramen Santouka    0.984303    NaN     8.7   Ramen Restaurant   
3       Aperture Coffee Bar    0.927561    1.0     8.7               Café   
11             Gyu-Kaku BBQ    0.973334    3.0     8.7          BBQ Joint   
2   Elysian Coffee Broadway    0.979981    1.0     8.6        Coffee Shop   
17         The Cascade Room    0.921301    2.0     8.5        Music Venue   

     latitude   longitude  distance credit_card  beer outdoor_seating  
7   49.263877 -123.105307       674        True  True           False  
9   49.2

In [11]:
def mainfunction_yelp(llvalue):
    request_code = yelp_station(llvalue)
    point = POI_yelp(request_code)
    return df_yelp(point)

print(mainfunction_yelp("49.262487,-123.114397").sort_values(by='rating', ascending=False).head(10))

                   name price  rating   latitude  longtitude  review_count  \
9             Uma Sushi   $$$     4.5  49.263805 -123.113729           109   
48         Simit Bakery  None     4.5  49.263235 -123.108772            33   
39            Deja Brew    $$     4.5  49.263660 -123.125130            52   
38     Yu Shang Hot Pot  None     4.5  49.263149 -123.125629            66   
8   Indian Roti Kitchen    $$     4.5  49.259010 -123.115380           154   
19  Mangia Cucina & Bar    $$     4.5  49.265427 -123.106907            72   
27           Maizal RMF    $$     4.5  49.260010 -123.101299           141   
29            Superbaba  None     4.5  49.263330 -123.101268            67   
30          Menya Raizo    $$     4.0  49.263360 -123.113140            68   
21       Burgoo Bistros    $$     4.0  49.257630 -123.100840           486   

       distance  
9    152.157897  
48   401.706105  
39   789.664375  
38   818.363433  
8    391.283162  
19   634.271387  
27   985.254194

In [ ]:
#Change dataframe to csv
list_of_df = []
for indx in range(len(ll)):
    combined = mainfunction_foursq(ll[indx])
    combined['bikes'] = df['free_bikes'][indx]
    list_of_df.append(combined)

newdfs = pd.concat(list_of_df)
newdfs.to_csv('fs_restaurant_info.csv')